In [10]:
! pip install pandas
! pip install pydicom
! pip install seaborn

import glob, pylab, pandas as pd
import pydicom, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from skimage.transform import resize
from skimage.exposure import equalize_hist


import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
sns.set_palette(sns.dark_palette("purple"))

import torch
cuda = torch.cuda.is_available()

from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from functools import reduce

import torch.nn as nn
from torch.nn.functional import softplus
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, Dropout2d, MaxPool2d, BatchNorm1d, BatchNorm2d, ReLU, ELU
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax, dropout, dropout2d
import time

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
# define size variables
IMG_SIZE = 224
height = IMG_SIZE
width = IMG_SIZE
channels = 1
num_features = 224**2

# Regulization
L2_reg = 1e-6

# 1. Conv Layer
conv_out_channels = [8, 16, 32, 64]
conv_kernel = [15, 11, 5, 5]
conv_padding = [7, 5, 2, 2]
conv_stride = [2, 2, 1, 1]

# 1. MaxPool Layer
pool_kernel = 4
pool_padding = 2
pool_stride = 2





######## Image has to be: (num, channels, height, width)!!!! #########
class CNN_VAE(nn.Module):
    # Calculating the dimensions 
    def compute_conv_dim(height, width, kernel_size, padding_size, stride_size):
        height_new = int((height - kernel_size + 2 * padding_size) / stride_size + 1)
        width_new =  int((width  - kernel_size + 2 * padding_size) / stride_size + 1)
        return [height_new, width_new]

    def compute_final_dimension(height, width, last_num_channels, num_layers):
        # First conv layer
        CNN_height = height
        CNN_width = width
        for i in range(num_layers):
            # conv layer
            CNN_height, CNN_width = compute_conv_dim(CNN_height, CNN_width, conv_kernel[i], conv_padding[i], conv_stride[i])
            # maxpool layer
            CNN_height, CNN_width = compute_conv_dim(CNN_height, CNN_width, pool_kernel, pool_padding, pool_stride)
        final_dim = CNN_height * CNN_width * last_num_channels
    return final_dim

    def __init__(self, latent_features, num_samples):
        super(CNN_VAE, self).__init__()
        
        self.latent_features = latent_features
        self.num_samples = num_samples
        
        # Calculate final size of the CNN
        self.final_dim = compute_final_dimension(height,width,conv_out_channels[-1],4)
        print(self.final_dim)
        ## CNN encoder
        self.encoder = nn.Sequential(            
                Conv2d(     in_channels=channels,
                            out_channels=conv_out_channels[0],
                            kernel_size=conv_kernel[0],
                            stride=conv_stride[0],
                            padding=conv_padding[0]),
            
                MaxPool2d(  kernel_size=pool_kernel, 
                            stride=pool_stride,
                            padding=pool_padding),
                ReLU(),
                BatchNorm2d(conv_out_channels[0]),
                Dropout2d(p=0.2),
            
                Conv2d(     in_channels=channels,
                            out_channels=conv_out_channels[1],
                            kernel_size=conv_kernel[1],
                            stride=conv_stride[1],
                            padding=conv_padding[1]),
            
                MaxPool2d(  kernel_size=pool_kernel, 
                            stride=pool_stride,
                            padding=pool_padding),
                ReLU(),
                BatchNorm2d(conv_out_channels[1]),
                Dropout2d(p=0.2),
            
                Conv2d(     in_channels=channels,
                            out_channels=conv_out_channels[2],
                            kernel_size=conv_kernel[2],
                            stride=conv_stride[2],
                            padding=conv_padding[2]),
            
                MaxPool2d(  kernel_size=pool_kernel, 
                            stride=pool_stride,
                            padding=pool_padding),
                ReLU(),
                BatchNorm2d(conv_out_channels[2]),
                Dropout2d(p=0.2),
                
                Conv2d(     in_channels=channels,
                            out_channels=conv_out_channels[3],
                            kernel_size=conv_kernel[3],
                            stride=conv_stride[3],
                            padding=conv_padding[3]),
            
                MaxPool2d(  kernel_size=pool_kernel, 
                            stride=pool_stride,
                            padding=pool_padding),
                ReLU(),
                BatchNorm2d(conv_out_channels[3]),
                Dropout2d(p=0.2),

        )
        
        self.CNN_to_latent = Linear(in_features=self.final_dim, out_features=self.latent_features*2)
        
        # The latent code must be decoded into the original image
        self.decoder = nn.Sequential(
            Linear(in_features=self.latent_features, out_features=128),
            #BatchNorm1d(num_features=128, eps=1e-3),
            ELU(),
            Linear(in_features=128, out_features=256),
            #BatchNorm1d(num_features=256),
            ELU(),
            Linear(in_features=256, out_features=num_features)
        )
        

    def forward(self, x): 
        outputs = {}
        
        x = self.encoder(x)
        batch_size = x.size(0)
        x = x.view( batch_size, -1)
        x = self.CNN_to_latent(x)
        
        # Split encoder outputs into a mean and variance vector
        mu, log_var = torch.chunk(x, 2, dim=-1)
        
        # Make sure that the log variance is positive
        log_var = softplus(log_var)
        
        # :- Reparametrisation trick
        # a sample from N(mu, sigma) is mu + sigma * epsilon
        # where epsilon ~ N(0, 1)
                
        # Don't propagate gradients through randomness
        with torch.no_grad():
            batch_size = mu.size(0)
            epsilon = torch.randn(batch_size, self.num_samples, self.latent_features)
            
            if cuda:
                epsilon = epsilon.cuda()
        
        sigma = torch.exp(log_var/2)
        
        # We will need to unsqueeze to turn
        # (batch_size, latent_dim) -> (batch_size, 1, latent_dim)
        z = mu.unsqueeze(1) + epsilon * sigma.unsqueeze(1)  
        
        # Run through decoder
        x = self.decoder(z)
        
        # The original digits are on the scale [0, 1]
        x = torch.sigmoid(x)
        
        # Mean over samples
        x_hat = torch.mean(x, dim=1)
        
        # Resize x_hat from [batch_size, no_features] to [batch_size, channels, height, width]
        x_hat = x_hat.view( batch_size, 1, height, width)
        
        outputs["x_hat"] = x_hat
        outputs["z"] = z
        outputs["mu"] = mu
        outputs["log_var"] = log_var
        
        return outputs


latent_features = 4
# The number of samples used then initialising the VAE, 
# is number of samples drawn from the distribution
num_samples = 10

net = CNN_VAE(latent_features, num_samples)

# Transfer model to GPU if available
if cuda:
    net = net.cuda()


57 57
15 15
8 8
5 5
1600
CNN_VAE(
  (encoder): Sequential(
    (0): Conv2d(1, 8, kernel_size=(15, 15), stride=(2, 2), padding=(7, 7))
    (1): MaxPool2d(kernel_size=4, stride=2, padding=2, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.2)
    (5): Conv2d(1, 16, kernel_size=(11, 11), stride=(2, 2), padding=(5, 5))
    (6): MaxPool2d(kernel_size=4, stride=2, padding=2, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.2)
    (10): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (11): MaxPool2d(kernel_size=4, stride=2, padding=2, dilation=1, ceil_mode=False)
    (12): ReLU()
    (13): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout2d(p=0.2)
    (15): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=

In [ ]:
from Network import CNN